In [1]:
# 데이터 로드
import gdown
file_id = '1EMmSiot45KKtt5OpqKPbfbEri-5T6q6w'
download_url = f'https://drive.google.com/uc?id={file_id}'
gdown.download(download_url, 'data.zip', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1EMmSiot45KKtt5OpqKPbfbEri-5T6q6w
To: /content/data.zip
100%|██████████| 2.60M/2.60M [00:00<00:00, 35.2MB/s]


'data.zip'

In [2]:
!unzip data.zip

Archive:  data.zip
  inflating: nsmc_train_label.npy    
  inflating: nsmc_test_input.npy     
  inflating: nsmc_test_label.npy     
  inflating: nsmc_train_input.npy    


In [3]:
import numpy as np

In [5]:
input_data = np.load('/content/nsmc_train_input.npy')
label_data = np.load('/content/nsmc_train_label.npy')

In [6]:
input_data

array([[  463,    20,   265, ...,     0,     0,     0],
       [  604,     1,   219, ...,   969,   680,    24],
       [  393,  2456, 25028, ...,     2,   226,    13],
       ...,
       [   60,  1458,  5612, ..., 10480, 10415,  1050],
       [ 1046,     1,  2271, ...,  1231,   545,  7862],
       [    1,  1806,    22, ...,    63,  1290,     1]], dtype=int32)

In [7]:
# 데이터 분리 학습용 검증용
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,v_val = train_test_split(input_data, label_data, test_size=0.2, random_state=42)

In [9]:
# 하이퍼 파라메터
model_name = 'cnn_classifier_kr'
batch_size = 128
num_epochs = 5
learning_rate = 0.001
max_len = x_train.shape[1]
vocab = 10000
kargs = {'model_name': model_name,
         'vocab': vocab,
         'max_len': max_len,
         'batch_size': batch_size,
         'num_epochs': num_epochs,
         'num_filters' : 100,
         'dropout_rate' : 0.5,
         'hidden_dim' : 250,
         'output_dim' : 1,
         'lr' : learning_rate}

In [11]:
import tensorflow as tf
class CNN_Classfier(tf.keras.Model):
  def __init__(self, **kargs):
    super(CNN_Classfier, self).__init__()
    self.embedding = tf.keras.layers.Embedding(input_dim=kargs['vocab'], output_dim=100)
    self.conv_list = [tf.keras.layers.Conv1D(filters=kargs['num_filters'],
                                             kernel_size=kernel_size, padding='valid', activation='relu')
                                              for kernel_size in [3,4,5] ]
    self.pooling = tf.keras.layers.MaxPool1D()
    self.dropout = tf.kearas.layers.Dropout(kargs['dropout_rate'])
    # kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.) 가중치를 이전상태와 비스하게 만들어서 과적합을 방지
    self.fc1 = tf.keras.layers.Dense(units=kargs['hidden_dim'], activation='relu',
                                     kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
    self.fc2 = tf.keras.layers.Dense(units=kargs['output_dim'], activation='sigmoid',
                                     kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
  def call(self, x):
    x = self.embedding(x)
    x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)  # axis=-1 마지막 축을 기준으로 붙인다
    x = self.fc1(x)
    x = self.fc2(x)
    return x

In [ ]:
# 모델정의
# 컴파일
# 콜벡(필요시)
# 모델학습
# 평가